In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
from PIL import Image, ImageDraw
from PIL.ImageOps import invert

In [2]:
def monochrome(input_array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array

In [3]:
def binarization(input_array):
    height, width = input_array.shape[:2]
    input_array1 = monochrome(input_array)
    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array1[i][j] <= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0

    return output_array


In [4]:
def first_nonzero(arr):
    try:  return np.nonzero(arr)[0][0]
    except IndexError:  return -1

In [5]:
def last_nonzero(arr):
    try:
        return np.shape(arr)[0] - np.nonzero(np.flip(arr))[0][0] - 1
    except IndexError:
        return -1

In [6]:
def profile_y(input_array):
    output_array2 = input_array
    return {'y_axis' : np.arange(0, output_array2.shape[0]).astype(int), # разметка оси y, 0 тк первое значение размера это колво строк
            'values' : np.sum(output_array2, axis=1) # проекция на ось y
            }

In [7]:
def profile_x(input_array):
    output_array1 = input_array
    return {'values' : np.sum(output_array1, axis=0), # проекция на ось x
            'x_axis' : np.arange(0, output_array1.shape[1]).astype(int) # разметка оси x
            }

In [8]:
def X_(input_array):
    height, width = input_array.shape[:2]
    p = profile_x(input_array)

    plt.bar(x=p['x_axis'], height=p['values'], width=0.85)
    plt.ylim(0, max(p['values']))
    plt.xlim(0, max(p['x_axis']))
    plt.savefig("x_profile.png")
    plt.close()

In [9]:
def Y_(input_array):
    height, width = input_array.shape[:2]
    p = profile_y(input_array)

    plt.barh(y=p['y_axis'], width=p['values'], height=0.85)
    plt.ylim(max(p['y_axis']), 0)
    plt.xlim(0, max(p['values']))
    plt.savefig("y_profile_y.png")
    plt.close()


In [10]:
def edit_img(img):
    img_array = np.array(img)

    img_array_01 = binarization(img_array)
    img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    return img_array_cut

In [11]:
def part_blank_regions(input_array):
    height, width = input_array.shape[:2]

    top = 0
    bottom = height - 1
    while top < height:
        row = input_array[top]
        if np.any(row != 255):
            break
        top += 1
    while bottom >= 0:
        row = input_array[bottom]
        if np.any(row != 255):
            break
        bottom -= 1

    left = 0
    right = width - 1
    while left < width:
        col = input_array[top:bottom+1, left]
        if np.any(col.flatten() != 255):
            break
        left += 1
    while right >= 0:
        col = input_array[top:bottom+1, right]
        if np.any(col.flatten() != 255):
            break
        right -= 1

    cropped = input_array[top:bottom+1, left:right+1]

    new_image = Image.fromarray(cropped)
    new_image.save('1_cropped.png')

In [12]:
def segmentation(input_array):
    p = profile_x(input_array)
    height, width = input_array.shape[:2]

    borders = {'left': [], 'right': [], 'up': [], 'down': []}

    for i in range(p['x_axis'].shape[0] - 1):
        if p['values'][i] == 0 and p['values'][i + 1] > 0:
            borders['left'].append(i)
        elif p['values'][i] > 0 and p['values'][i + 1] == 0:
            borders['right'].append(i)

    if borders['left'] and borders['left'][0] > borders['right'][0]:
        borders['left'].insert(0, 0)
    if borders['right'] and borders['right'][-1] < width - 1:
        borders['right'].append(width - 1)

    borders['left'].sort()
    borders['right'].sort()

    for left, right in zip(borders['left'], borders['right']):
        if left < right:
            segment = input_array[:, left:right]
            py = profile_y(segment)
            up = first_nonzero(py['values'])
            down = last_nonzero(py['values'])
            borders['up'].append(up)
            borders['down'].append(down)

    return borders


In [13]:
def draw_brders(input_image, list_borders):
    img_array = np.array(input_image)
    img = Image.fromarray(img_array)
    draw = ImageDraw.Draw(img)

    for k in range(len(list_borders['left'])):
        x1, x2 = list_borders['left'][k], list_borders['right'][k]
        y1, y2 = list_borders['up'][k], list_borders['down'][k]
        draw.rectangle([(x1, y1), (x2, y2)], outline='orange', width=2)

    img.save('segmentation.png')

In [14]:
def errosion(input_array):

    height, width = input_array.shape[:2]
    output_array = input_array.copy()
    for i in range (1, width-1):
        for j in range (1, height-1):
            if input_array[j][i] == 1 and (input_array[j][i+1] == 0 or input_array[j][i-1] == 0):
                output_array[j][i] = 0
            else:
                output_array[j][i] = input_array[j][i]

    return output_array

In [15]:
def no_tilt(input_array, num):
    print(input_array.shape)
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for x in range(width):
        for y in range(height):

            a = y*num
            i, d = divmod(a, 1)
            if i+x < width:
                output_array[y][x] = input_array[y][int(i+x)] *255
    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('1_no_tilt.bmp')
    return output_array

In [16]:
def main():

    input_image = Image.open('1.bmp')
    input_array = np.array(input_image)

    part_image = part_blank_regions(input_array)

    input_image = Image.open('1_cropped.png')
    input_array = np.array(input_image)


    segment = edit_img(input_array)

    X_(segment)
    Y_(segment)

    segment = errosion(segment)
    segment = no_tilt(segment, -0.23)


    list = segmentation(segment)
    draw_brders(input_image , list)


if __name__ == '__main__':
    main()

(31, 807)
